In [59]:
import numpy as np
import pandas as pd

In [60]:
class Interviewer:
    def __init__(self, name: str, rate_per_hour: float):
        """
        :param rate_per_hour: float, ставка інтерв'юера (грн/година)
        """

        self.name: str = name
        self.rate_per_hour = rate_per_hour
        self._total_hours = 0
        
    def add_worked_hours(self, hours: float):
        self._total_hours += hours
    
    @property
    def salary(self) -> float:
        return self._total_hours * self.rate_per_hour


class SurveyModel:
    def __init__(
            self,
            respondent_average_appearance_time: float,
            survey_average_response_time: float,
            positive_probability: float,
            interviewer: Interviewer,
    ):
        """
        :param respondent_average_appearance_time: float, середній час появи респондента (секунди)
        :param survey_average_response_time: float, середній час одного інтерв'ю (секунди)
        :param positive_probability: float, ймовірність відповіді "так" (0.0 - 1.0)
        """

        self.avg_respondent_appearance_time = respondent_average_appearance_time
        self.avg_response_time = survey_average_response_time
        self.positive_probability = positive_probability
        self.interviewer = interviewer

    def simulate(self, respondents_count: int) -> pd.DataFrame:
        respondent_appearance_time = np.random.exponential(scale=self.avg_respondent_appearance_time, size=respondents_count)
        respondent_response_time = np.random.exponential(scale=self.avg_response_time, size=respondents_count)
        respondent_probability = np.random.choice(
            [1, 0], size=respondents_count, p=[self.positive_probability, 1 - self.positive_probability]
        )

        total_appearance_time = np.cumsum(respondent_appearance_time)
        total_end_time = np.zeros(respondents_count)
        respondent_is_taken_survey = np.zeros(respondents_count, dtype=int)

        survey_end_time = 0
        
        for i in range(respondents_count):
            if respondent_probability[i] and total_appearance_time[i] >= survey_end_time:
                respondent_is_taken_survey[i] = 1
                survey_end_time = total_appearance_time[i] + respondent_response_time[i]
                total_end_time[i] = survey_end_time
            else:
                total_end_time[i] = None
        
        print(survey_end_time)
        survey_end_time = survey_end_time / 60 / 60 
        self.interviewer.add_worked_hours(survey_end_time)
        print(
            f"Загальний час проведення інтерв'ю: {survey_end_time:.2f} годин. "
            f"Інтерв'ювер {self.interviewer.name} заробив {self.interviewer.rate_per_hour * survey_end_time:.2f} грн."
        )

        return pd.DataFrame(
            data={
                "respondent_appearance_time": respondent_appearance_time,
                "respondent_response_time": respondent_response_time,
                "respondent_probability": respondent_probability,
                "respondent_came_time": total_appearance_time,
                "respondent_survey_end_time": total_end_time,
                "respondent_is_taken_survey": respondent_is_taken_survey,
            },
            index=[f"respondent_{i}" for i in range(1, respondents_count + 1)]
        )

In [61]:
model = SurveyModel(
    respondent_average_appearance_time=60,
    survey_average_response_time=100,
    positive_probability=0.7,
    interviewer=Interviewer(name='Serhii', rate_per_hour=500)
)

df = model.simulate(respondents_count=100)
df

5799.861856082329
Загальний час проведення інтерв'ю: 1.61 годин. Інтерв'ювер Serhii заробив 805.54 грн.


,respondent_appearance_time,respondent_response_time,respondent_probability,respondent_came_time,respondent_survey_end_time,respondent_is_taken_survey
respondent_1,39.143805,5.477840,1,39.143805,44.621645,1
respondent_2,154.452496,60.983222,0,193.596301,NaN,0
respondent_3,20.860205,75.038965,1,214.456506,289.495470,1
respondent_4,108.337252,5.495822,1,322.793758,328.289580,1
respondent_5,1.342144,50.916624,1,324.135902,NaN,0
...,...,...,...,...,...,...
respondent_96,10.744777,854.222044,1,4945.639812,5799.861856,1
respondent_97,169.457272,208.577592,1,5115.097085,NaN,0
respondent_98,11.917919,135.082282,1,5127.015003,NaN,0
respondent_99,45.247485,79.220538,1,5172.262488,NaN,0
